# Molecular Dynamics Coursework

In [1]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

The task is a straightforward (!) continuation of the final part of the lab 2 worksheet, and is based on [the same paper as used there](http://dx.doi.org/10.1021/jp046158d).

## Water

Take the potential given for *multiple* water molecules

$$
\begin{align}
  V(\vec{r}) & = \sum_{k=1}^N \left\{ \sum_{l = 1}^2 D_0 \left[ 1 - e^{\alpha \Delta r_{OH_{l_k}}} \right]^2 + \frac{1}{2} k_{\theta} \Delta r_{HH_k}^2 + k_{r \theta} \Delta r_{HH_k} \left( \Delta r_{OH_{1_k}} + \Delta r_{OH_{2_k}} \right) + k_{rr} \Delta r_{OH_{1_k}} \Delta r_{OH_{2_k}} \right\} + \\ & \quad \sum_{i>j} \left( \frac{e_i e_j}{4 \pi e_0 r_{ij}} + 4 \epsilon_{ij} \left[ \left( \frac{\sigma_{ij}}{r_{ij}} \right)^{12} - \left( \frac{\sigma_{ij}}{r_{ij}} \right)^6 \right] \right).
\end{align}
$$

Here $k$ sums over the different water *molecules*, of which there are $N$. The first term corresponds to the forces internal to any one individual molecule. The $i, j$ indices sum over all *atoms*, and correspond to the forces coupling the atoms in different molecules.

From the lab 2 sheet, we have that the forces within the molecule on the appropriate atoms are, eg,

$$
\begin{align}
  \vec{F}_{O} & = - \frac{\partial V}{\partial \vec{r}_O} \\
  & = - \left\{ -2 D_0 \sum_{l=1}^2 \left[ 1 - e^{\alpha \Delta r_{OH_l}} \right] \frac{\partial \Delta r_{OH_l}} {\partial \vec{r}_O} \alpha e^{\alpha \Delta r_{OH_l}} + \right. \\ & \qquad \left. k_{r \theta} \Delta r_{HH} \left( \frac{\partial \Delta r_{OH_1}}{\partial \vec{r}_O} + \frac{\partial \Delta r_{OH_2}}{\partial \vec{r}_O} \right) + \right. \\ & \qquad\left. k_{rr} \left( \frac{\partial \Delta r_{OH_1}}{\partial \vec{r}_O}  \Delta r_{OH_2} + \Delta r_{OH_1} \frac{\partial \Delta r_{OH_2}}{\partial \vec{r}_O} \right) \right\}, \\
  \vec{F}_{H_1} & = -\left\{  -2 D_0 \left[ 1 - e^{\alpha \Delta r_{OH_1}} \right] \frac{\partial \Delta r_{OH_l}}{\partial \vec{r}_{H_1}}  \alpha e^{\alpha \Delta r_{OH_1}} + \right. \\& \qquad \left. k_{\theta} \Delta r_{HH} \frac{\partial \Delta r_{HH}}{\partial \vec{r}_{H_1}} + k_{r \theta} \left[ \frac{\partial \Delta r_{HH}}{\partial \vec{r}_{H_1}} \left( \Delta r_{OH_1} + \Delta r_{OH_2} \right) + \Delta r_{HH} \frac{\partial r_{OH_1}}{\partial \vec{r}_{H_1}} \right] + \right. \\ & \qquad \left. k_{rr} \Delta r_{OH_2} \frac{\partial \Delta r_{OH_1}}{\partial \vec{r}_{H_1}} \right\}
\end{align}
$$

The total force of any atom in addition picks up a term from the Lennard-Jones interaction with all other atoms, with all atoms intereacting through the electronegativities $e_i$, and the oxygen atoms interacting through the Lennard-Jones potential, leading to the additional force on atom $j$ of

$$
\begin{equation}
  \vec{F}_j = \sum_{i>j} \left\{-\frac{e_i e_j}{4 \pi e_0 r_{ij}^3}  + 24 \frac{\epsilon_{ij}}{r_{ij}^2} \left[ 2 \left( \frac{\sigma_{ij}}{r_{ij}} \right)^{12} - \left( \frac{\sigma_{ij}}{r_{ij}} \right)^6  \right] \right\} \left( \vec{r}_i - \vec{r}_j \right).
\end{equation}
$$

The definitions of the terms are the same as in lab 2: $\Delta r_{OH_l} = r_{OH_l} - r_{OH_{\text{eq}}}$ is the stretch in the distance between the oxygen atom and the $l^{\text{th}}$ hydrogen atom $r_{OH_l}$ and its equilibrium value, and $\Delta r_{HH} = r_{HH_l} - r_{HH_{\text{eq}}}$ is the stretch in the distance between the hydrogen atoms. The parameter values are

1. $D_0 = 101.9188$ kcal/mol.
2. $\alpha = 2.567$ $Å^{-1}$
3. $k_{\theta} = 328.645606$ kcal/mol/$Å^2$
4. $k_{r \theta} = -211.4672$ kcal/mol/$Å^2$
5. $k_{rr} = 111.70765$ kcal/mol/$Å^2$
6. $r_{OH_{\text{eq}}} = 1Å$
7. $r_{HH_{\text{eq}}} = 1.633Å$
8. $e_H = 0.41 e_0$
9. $e_O = -0.82 e_0$
10. $\sigma_{OO} = 3.166Å$
11. $\epsilon_{OO} = 0.1554$ kcal/mol
12. $\epsilon_{OH} = \epsilon_{HH} = 0$ kcal/mol
13. $e_0 = 1$ (unit charge).

A cut-off radius $R_c = 14Å$ is enforced; the additional interaction term leading to the force $\vec{F}_j$ is only calculated inside this radius.

### Set up

Consider a periodic box setup as in lab 1. Each box will have $\vec{x} \in [0, 35Å]^3$.

Put eight water molecules in the box. The locations of the oxygen atoms should be drawn "randomly" from a uniform distribution in $[0, 35]$ with random seed set to `100`. 

The hydrogen atoms should be located at

$$
\begin{equation}
  \vec{r}_{H_{\pm_k}} = \vec{r}_{O_k} + [ \pm 0.8, 0.6, 0].
\end{equation}
$$

Plot the initial locations of the atoms. Starting from this configuration, minimize the energy (allowing the locations of the oxygen atoms to vary, but maintaining the offset of the hydrogen atoms within the molecule as fixed) and plot the resulting location of the atoms.

In [2]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12,6)
from scipy.optimize import minimize

### Evolution

Using a timestep $dt = 0.001$, evolve forward to $T = 5$ and plot the temperature as a function of time. [Note: when testing, you should only use a few timesteps. As a hint, the temperature should remain ${\cal O}(1)$ and oscillate $\sim 10$ times before $T=1$].